In [33]:
import pandas as pd
import numpy as np
import ast

In [3]:
link1 = "./BD/df_basic_name_clean.csv"
link2 = "./BD/df_t_akas_clean.csv"
link3 = "./BD/df_t_basics_clean.csv"
link4 = "./BD/df_t_crew_clean.csv"
link5 = "./BD/df_t_episode_clean.csv"
link6 = "./BD/df_t_principals_clean.csv"
link7 = "./BD/df_t_ratings_clean.csv"
link8 = "./BD/tmdb_full.csv"
link9 = "./BD/df_tID.csv"
link10 = "./BD/df_nID.csv"

In [79]:
df_basic_name = pd.read_csv(link1)

In [21]:
df_t_akas = pd.read_csv(link2)

In [14]:
df_t_basics = pd.read_csv(link3)

In [22]:
df_t_crew = pd.read_csv(link4)

In [ ]:
df_t_episode = pd.read_csv(link5)

In [4]:
df_t_principals = pd.read_csv(link6)

In [16]:
df_t_ratings = pd.read_csv(link7)

In [94]:
df_tmdb = pd.read_csv(link8, low_memory = False)

In [68]:
df_tID = pd.read_csv(link9)

In [90]:
df_nID = pd.read_csv(link10)

On veut 3 df: un df avec index tconst,un avec index nconst, et la table tmdb nettoyée et triée
1.  Commençons par df_tID qui donnera les réalisateurs et acteurs pour chaque films ainsi que tous les titres possibles pour la recherche

In [ ]:
# Prépartion table title basics pour trier les IDs de films pour la période souhaitée et supprimer les pornos:
df_tb_tconst = df_t_basics[(df_t_basics['titleType'] == 'movie') & (df_t_basics['startYear'] >= 1960) & (df_t_basics['isAdult'] == 0)]
df_tb_tconst = df_tb_tconst[['tconst']]

#Préparation table title principals pour afficher des listes d'acteurs pour chaque id de film:
df_ptconst = df_t_principals[(df_t_principals['category'] == 'actor') | (df_t_principals['category'] == 'actress')]
df_ptconst = df_ptconst.drop('category', axis = 1)
df_ptconst = df_ptconst .groupby('tconst').agg(list)

#Premier merge de title principal sur les tconst triés de title basic:
df_tID = pd.merge(df_tb_tconst, df_ptconst, how = 'left', on='tconst')

#2e merge avec akas deja nettoyé
df_tID = pd.merge(df_tID, df_t_akas, how = 'left', left_on='tconst', right_on='titleId')

#3e merge avec les directors de title crew
df_t_crew_fusion = df_t_crew[['tconst', 'directors']]
df_tID = pd.merge(df_tID, df_t_crew_fusion, how = 'left', on='tconst')

#Drop colonnes doublons
df_tID = df_tID.drop(['titleId'], axis = 1)

#Incorporation des noms des réalisateurs et acteurs:

#La colonne nconst a des listes, on veut remplacer les NaN par des listes vides:
df_tID['nconst'] = df_tID['nconst'].apply(lambda x: x if isinstance(x, list) else [])

#les colonnes directors et title sont des strings de listes, on remplace donc d'abordles NaN 
#par des strings de liste vide avant de tout changer en listes :
df_tID['directors'] = df_tID['directors'].fillna('[]')
df_tID['title'] = df_tID['title'].fillna('[]')

#On remplace les strings de listes par des listes
df_tID['directors'] = df_tID['directors'].apply(ast.literal_eval)
df_tID['title'] = df_tID['title'].apply(ast.literal_eval) 

#On utilise la table basic name qui associe un ID a un nom et on les réunis en dico {ID : nom}:
dico = dict(zip(df_basic_name["nconst"], df_basic_name["primaryName"]))

#On défini une fonction pour faire ce que l'on cherche et on applique:
def id_nom(x):
    rendu = []
    for id in x:
        if id in dico.keys(): #Il y a quelques id inconnus je suis obligé de mettre une condition
            rendu.append(dico[id])
    return rendu

df_tID["Nom_Réalisateurs"] = df_tID["directors"].apply(id_nom)
df_tID["Nom_acteurs"] = df_tID['nconst'].apply(id_nom)

#On remet dans l'ordre:
df_tID= df_tID[['tconst', 'Nom_Réalisateurs', 'Nom_acteurs', 'title', 'directors', 'nconst']]

#Renommage des colonnes
df_tID = df_tID.rename(columns={'tconst' : 't_ID', 'nconst' : 'nID_acteurs', 'title' : 'titres_possibles',  'directors' : 'nID_réalisateurs'})

#reset index
df_tID= df_tID.reset_index().drop('index', axis = 1)

#Export
df_tID.to_csv('df_tID.csv', index = False)

2. Ensuite un df_nID

In [ ]:
#Tri de la table title basic pour avoir les tconst correspondant aux films
df_tb_nconst = df_tb_tconst = df_t_basics[(df_t_basics['titleType'] == 'movie') & (df_t_basics['startYear'] >= 1960) & (df_t_basics['isAdult'] == 0)]
df_tb_nconst = df_tb_nconst[['tconst']]
df_tb_nconst = df_tb_nconst.reset_index()

# Préparation table title principals avec seulement les acteurs
df_t_principals_fusion = df_t_principals[(df_t_principals['category'] == 'actor') | (df_t_principals['category'] == 'actress')]
df_t_principals_fusion = df_t_principals_fusion.drop('category', axis = 1)

#Premier groupby tconst pour merge avec les tconst triés de title basics
df_t_principals_fusion.groupby('tconst').agg(list).reset_index()

#Premier merge par tconst
df_nID = pd.merge(df_t_principals_fusion, df_tb_nconst, how = 'left', on = 'tconst')

#Explode de la table mergée pour refaire le bon groupby cette fois par nconst
df_nID = df_nID.explode('nconst')
df_nID = df_nID.groupby('nconst').agg(list).reset_index()
df_nID = df_nID.drop('index', axis = 1)

#Ajouter des colonnes d'infos de l'acteur et une colonne nb de films:
df_basic_name_fusion = df_basic_name[['nconst', 'primaryName', 'birthYear', 'deathYear']]
df_nID = pd.merge(df_nID, df_basic_name_fusion, how = 'left', on = 'nconst')
df_nID['Nb_films'] = df_nID['tconst'].apply(lambda x : len(x))

#On renomme les colonnes
df_nID = df_nID.rename(columns={'nconst' : 'n_ID', 'tconst' : 'films_joués', 'primaryName' : 'Nom', 'birthYear' : 'Année_naissance', 'deathYear' : 'Année_décès'})

#On remet bien 'films_joués' en liste!
df_nID["films_joués"] = df_nID["films_joués"].apply(ast.literal_eval) 

#Plus qu'a exporter!
df_nID.to_csv('df_nID.csv', index = False)

In [92]:
df_nID

,n_ID,films_joués,Nom,Année_naissance,Année_décès,Nb_films
0,nm0000001,"[tt0023926, tt0024025, tt0025164, tt0026942, t...",Fred Astaire,1899.0,1987.0,64
1,nm0000002,"[tt0037382, tt0037610, tt0038355, tt0039302, t...",Lauren Bacall,1924.0,2014.0,79
2,nm0000003,"[tt0044881, tt0046200, tt0047607, tt0048001, t...",Brigitte Bardot,1934.0,NaN,39
3,nm0000004,"[tt0076816, tt0077147, tt0077621, tt0077975, t...",John Belushi,1949.0,1982.0,105
4,nm0000005,"[tt0043048, tt0064793, tt0064897, tt0069467, t...",Ingmar Bergman,1918.0,2007.0,11
...,...,...,...,...,...,...
3432237,nm9993700,[tt8744178],Sexy Angel,NaN,NaN,1
3432238,nm9993701,[tt8736744],Sanjai Kuriakose,NaN,NaN,1
3432239,nm9993703,"[tt11212278, tt26598427]",James Craigmyle,NaN,NaN,2
3432240,nm9993708,"[tt11697102, tt11702702, tt11772842, tt1450881...",Eli Bevins,NaN,NaN,5


3. Enfin nettoyage de la table TMDB qui sera notre base pour le système de recommandation

### Supprimer les lignes avec liste vide dans colonne genre
### Choisir les colonnes à garder

In [95]:
#L'énoncé nous demandant d'afficher l'affiche du film, on commence par supprimer les lignes qui ne renseignent pas le lien de l'affiche (45413/309572)
df_tmdb_clean = df_tmdb.dropna(subset=['poster_path'])

#Il reste 1 ligne qui renvoie à un film pour public adulte, restons courtois et supprimons la!
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean['adult'] == False]

#D'après notre étude de marché nous allons nous limiter aux films Français et Anglais, en faisant le filtre au niveau de la langue
#pour garder les films francophones et anglophones plutôt que français et anglais.
df_tmdb_clean = df_tmdb_clean[(df_tmdb_clean['original_language'] == 'fr') | (df_tmdb_clean['original_language'] == 'en')]
df_tmdb_clean = df_tmdb_clean[(df_tmdb_clean['spoken_languages'].str.contains('fr')) | (df_tmdb_clean['spoken_languages'].str.contains('en'))]

#La colonne 'release_date' doit être au format datetime pour filtrer les films > 1960 (suppression de 27335 lignes sur 264158 (10%)) et les classer:
df_tmdb_clean["release_date"] = pd.to_datetime(df_tmdb_clean["release_date"])
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean["release_date"].dt.year > 1960]
df_tmdb_clean = df_tmdb_clean.sort_values('release_date')

#On supprime 3600 lignes avec runtime = 0:
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean['runtime'] > 0]

#117 films pas encore sortis, on peut les retirer: 
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean['status'] == 'Released']

#Filtrage des lignes avec genre non renseigné
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean['genres'].str.len() > 2]
 
#Conversion de la colonne 'genre' en liste
import ast
df_tmdb_clean['genres'] = df_tmdb_clean['genres'].apply(ast.literal_eval) 

#remplacement des 0 par des NaN, puis des NaN par la moyenne des notes (meilleure car plus basse que la médiane et on ne la veux pas trop haute)
df_tmdb_clean['vote_average2'] = df_tmdb_clean['vote_average'].apply(lambda x : np.nan if x == 0 else x) 
moy = round(df_tmdb_clean[df_tmdb_clean['vote_average'] > 0]['vote_average'].mean(), 1)
df_tmdb_clean['vote_average'] = df_tmdb_clean['vote_average2'].fillna(moy)

df_tmdb_clean = df_tmdb_clean.reset_index()

#On va maintenant sélectionner et réaranger les colonnes pertinentes:
#INDEX on vire car on en fera un propre
#ADULT non pertinent car on ne les garde pas
#BACKDROP_PATH non pertinent car on peut avoir les posters
#Seulement 22316/236823 avec un BUDGET > 0 : colonne non pertinente pour ML mais on garde pour afficher si besoin
#genres on garde
#Homepage Non pertinent pour ML mais on peut garder pour le site
#id inutile car on garde seulement celui de imdb
#imdb_id on garde pour les merges
#original_language interessant on garde
#original_title cool pour le site et pour garder un df clair
#overview sympa pour le site
#popularity génial
#poster_path top pour le site
#production_countries trop complexe a utiliser, on vire
#release_date super pour site
#revenue : Seulement 14537/236823 > 0 : colonne non pertinente 
#runtime très utile!
#spoken_language compliqué à utiliser
#status inutile car tous pareils après filtre
#tagline quasi tous vides ou mal rensignés, à virer
#inutile avec original title
#video inutile car booléen
#vote_average super !
#vote_count inutile surtout
#pruduction companies inutiles


df_tmdb_clean = df_tmdb_clean[['imdb_id', 'original_title', 'release_date', 'runtime', 'genres', 'original_language', 'vote_average', 'popularity', 'overview', 'poster_path', 'budget',]]

df_tmdb_clean.to_csv('df_tmdb_clean.csv', index = False)

In [98]:
df_tmdb_clean

,imdb_id,original_title,release_date,runtime,genres,original_language,vote_average,popularity,overview,poster_path,budget
0,tt0054823,Doctor Blood's Coffin,1961-01-01,92,[Horror],en,5.0,1.452,After being thrown out of medical school for e...,/7GkBT8cKPl1GhBuUnktJpjF1uV.jpg,0
1,tt0260050,Girls Beware,1961-01-01,10,"[Crime, Drama]",en,5.0,0.741,Girls Beware is a trilogy of tragedies brought...,/iuMzdDCH2gJaVGGYaizzajvEyxO.jpg,0
2,tt0054886,Follow That Man,1961-01-01,84,[Comedy],en,5.8,0.841,A farcical comedy about a con-man and a girl r...,/xxcZ014zEsJm7WW8k5urHEhr783.jpg,0
3,tt0054807,The Devil's Messenger,1961-01-01,72,[Horror],en,4.3,1.130,In this feature version of the Swedish TV seri...,/yBfnRV6YNrZM33o1dwdjdQSoZGl.jpg,0
4,tt0054996,House of Mystery,1961-01-01,56,"[Mystery, Horror]",en,6.6,0.961,A young couple go to check out a house they're...,/9YVj1dwT4TXM59rytnAoCeySjWO.jpg,0
...,...,...,...,...,...,...,...,...,...,...,...
91330,tt17527468,Bottoms,2023-08-25,92,[Comedy],en,5.8,6.615,Two unpopular queer girls in their senior year...,/yiiCaamwKb5p2kD95jsjdGJRmTn.jpg,0
91331,tt26448981,"Orlando, ma biographie politique",2023-10-11,98,[Documentary],fr,5.8,2.411,Virginia Woolf’s “Orlando” tells the story of ...,/dsEsYLBgo9pIumB3kgToDWuOg7x.jpg,0
91332,tt26347358,Notre corps,2023-10-11,168,[Documentary],fr,5.8,2.426,"In a Parisian public hospital, Claire Simon qu...",/oamkG0bRZbsVS27mOrMTbz6UYNY.jpg,980000
91333,tt15683734,It Lives Inside,2023-11-03,99,"[Drama, Horror, Thriller]",en,5.8,3.508,"Sam is desperate to fit in at school, rejectin...",/dpgI82d8CrCDqDdBQeDUeFOIzI4.jpg,0
